<h1>Ensaio para Active Voxel</h1>

In [270]:
def console_log( item ):
    print( item )
    

In [271]:
# quads always start with 6 quads needed to form the seed
# <(( int e1, int e2 ) t1, ( int e3, int e4 ) t2 ), int >
quads = {
    # front
    (( 1, 2 ), ( 3, 4 )): 0,

    # top
    (( -3, 5 ), ( 6, 7 )): 1,

    # back
    (( -6, 8 ), ( 9, 10 )): 2,

    # bottom
    (( -9, 11 ), ( -1, 12 )): 3,

    # left
    (( -11, -8 ), ( -5, -2 )): 4,

    # right
    (( -12, -4 ), ( -7, -10 )): 5
}

console_log( quads )

{((1, 2), (3, 4)): 0, ((-3, 5), (6, 7)): 1, ((-6, 8), (9, 10)): 2, ((-9, 11), (-1, 12)): 3, ((-11, -8), (-5, -2)): 4, ((-12, -4), (-7, -10)): 5}


In [272]:
quads_rev = {}

# dict comprehension failed here
for quad in quads:
    quads_rev[ quads[ quad ]] = quad
    
console_log( quads_rev )

{0: ((1, 2), (3, 4)), 1: ((-3, 5), (6, 7)), 2: ((-6, 8), (9, 10)), 3: ((-9, 11), (-1, 12)), 4: ((-11, -8), (-5, -2)), 5: ((-12, -4), (-7, -10))}


In [273]:
edges_quads = {
    1: ( 0, 3 ),  2: ( 0, 4 ),  3: ( 0, 1 ),  4: ( 0, 5 ),
    5: ( 1, 4 ),  6: ( 1, 2 ),  7: ( 1, 5 ),  8: ( 2, 4 ),
    9: ( 2, 3 ), 10: ( 2, 5 ), 11: ( 3, 4 ), 12: ( 3, 5 )
}

console_log( edges_quads )

{1: (0, 3), 2: (0, 4), 3: (0, 1), 4: (0, 5), 5: (1, 4), 6: (1, 2), 7: (1, 5), 8: (2, 4), 9: (2, 3), 10: (2, 5), 11: (3, 4), 12: (3, 5)}


In [274]:
scale = 1.0

vertices = {
    # front
    # edge 1 = vx 1 -> 2
    1: ( scale, 0, 0, False ),     # vx 1
    2: ( 0, 0, 0, False ),         # vx 2

    # edge 2 = vx 2 (shared edge 1) -> 3
    3: ( 0, scale, 0, False ),     # vx 3

    # edge 3 = vx 3 (shared edge 2) -> 4
    4: ( scale, scale, 0, False ), # vx 4

    # edge 4 = vx 4 (shared edge 3) -> vx1 (shared edge 1) 

    # top
    # -1 * edge 3

    # edge 5 = vx 3 (shared edge 2) -> 5
    5: ( 0, scale, scale, False ), # vx 5

    # edge 6 = vx 5 (shared edge 5) -> 6
    6: ( scale, scale, scale, False ),# vx 6

    # edge 7 = vx 6 (shared edge 6) -> vx 4 (shared edge 3)

    # back
    # -1 * edge 6

    # edge 8 = vx 5 (shared edge 5) -> 7
    7: ( 0, 0, scale, False ),     # vx 7

    # edge 9 = vx 7 (shared edge 8) -> 8
    8: ( scale, 0, scale, False ) # vx 8

    # edge 10 = vx 8 (shared edge 9) -> vx 6 (shared edge 6)

    # bottom
    # -1 * edge 9
    # edge 11 = vx 7 (shared edge 8 ) -> vx 2 (shared edge 1 )
    # -1 * edge 1
    # edge 12 = vx 1 (shared edge 1 ) -> vx 8 (shared edge 9 )

    # left
    # -1 * edge 11, -1 * edge 8, -1 * edge 5, -1 * edge 2

    # right
    # -1 * edge 12, -1 * edge 4, -1 * edge 7, -1 * edge 10
}

console_log( vertices )

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False)}


In [275]:
# < int, List<( int edge, int t, int pos )>>

vx_to_edges = {
    1: [{ "edge": 1, "t": 1, "pos": 0 }, { "edge":  4, "t": 1, "pos": 1 }, { "edge": 12, "t": 1, "pos": 0 }],
    2: [{ "edge": 1, "t": 1, "pos": 1 }, { "edge":  2, "t": 1, "pos": 0 }, { "edge": 11, "t": 1, "pos": 1 }],
    3: [{ "edge": 2, "t": 1, "pos": 1 }, { "edge":  3, "t": 1, "pos": 0 }, { "edge":  5, "t": 1, "pos": 0 }],
    4: [{ "edge": 3, "t": 1, "pos": 1 }, { "edge":  4, "t": 1, "pos": 0 }, { "edge":  7, "t": 1, "pos": 1 }],
    5: [{ "edge": 5, "t": 1, "pos": 1 }, { "edge":  6, "t": 1, "pos": 0 }, { "edge":  8, "t": 1, "pos": 0 }],
    6: [{ "edge": 6, "t": 1, "pos": 1 }, { "edge":  7, "t": 1, "pos": 0 }, { "edge": 10, "t": 1, "pos": 1 }],
    7: [{ "edge": 8, "t": 1, "pos": 1 }, { "edge":  9, "t": 1, "pos": 0 }, { "edge": 11, "t": 1, "pos": 0 }],
    8: [{ "edge": 9, "t": 1, "pos": 1 }, { "edge": 10, "t": 1, "pos": 0 }, { "edge": 12, "t": 1, "pos": 1 }]
}

console_log( vx_to_edges )

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}]}


In [276]:
# <( int edge, int t, int pos ), int >

edge_to_vcs = {}

for vx in vx_to_edges:
    for edge in vx_to_edges[ vx ]:
        edge_to_vcs[ str( edge )] = vx
    
console_log( edge_to_vcs )


{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8}


In [277]:
# AvxEdges always starts with the 12 edges needed to build the seed
edges = {
    # front
    # edge 1 = vx 1 -> 2
    1:  { "start": 1, "end": 2, "parent": 1, "t": 1 },

    # edge 2 = vx 2 (shared edge 1) -> 3
    2: { "start": 2, "end": 3, "parent": 2, "t": 1 },

    # edge 3 = vx 3 (shared edge 2) -> 4
    3: { "start": 3, "end": 4, "parent": 3, "t": 1 },

    # edge 4 = vx 4 (shared edge 3) -> vx1 (shared edge 1) 
    4: { "start": 4, "end": 1, "parent": 4, "t": 1 },

    # top
    # edge 5 = vx 3 (shared edge 2) -> 5
    5: { "start": 3, "end": 5, "parent": 5, "t": 1 },

    # edge 6 = vx 5 (shared edge 5) -> 6
    6: { "start": 5, "end": 6, "parent": 6, "t": 1 },

    # edge 7 = vx 6 (shared edge 6) -> vx 4 (shared edge 3)
    7: { "start": 6, "end": 4, "parent": 7, "t": 1 },

    # back
    # edge 8 = vx 5 (shared edge 5) -> 7
    8: { "start": 5, "end": 7, "parent": 8, "t": 1 }, 

    # edge 9 = vx 7 (shared edge 8) -> 8
    9: { "start": 7, "end": 8, "parent": 9, "t": 1 },

    # edge 10 = vx 8 (shared edge 9) -> vx 6 (shared edge 6)
    10: { "start": 8, "end": 6, "parent": 10, "t": 1 },

    # bottom
    # edge 11 = vx 7 (shared edge 8 ) -> vx 2 (shared edge 1 )
    11: { "start": 7, "end": 2, "parent": 11, "t": 1 },

    # edge 12 = vx 1 (shared edge 1 ) -> vx 8 (shared edge 9 )
    12: { "start": 1, "end": 8, "parent": 12, "t": 1 }

    # left
    # -1 * edge 11, -1 * edge 8, -1 * edge 5, -1 * edge 2

    # right
    # -1 * edge 12, -1 * edge 4, -1 * edge 7, -1 * edge 2
}

edges_rev = {}

for _edge in edges:
    edges_rev[ str( edges[ _edge ]) ] = _edge 

    
console_log( edges )
console_log( edges_rev )

edges_children = {}


{1: {'start': 1, 'end': 2, 'parent': 1, 't': 1}, 2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}}
{"{'start': 1, 'end': 2, 'parent': 1, 't': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2, 't': 1}": 2, "{'start': 3, 'end': 4, 'parent': 3, 't': 1}": 3, "{'start': 4, 'end': 1, 'parent': 4, 't': 1}": 4, "{'start': 3, 'end': 5, 'parent': 5, 't': 1}": 5, "{'start': 5, 'end': 6, 'parent': 6, 't': 1}": 6, "{'start': 6, 'end': 4, 'parent': 7, 't': 1}": 7, "{'start': 5, 'end': 7, 'parent': 8, 't': 1}": 8, "{'start': 7, 'e

In [278]:
def quads_vertices():
    res = []
    
    for vx in vertices:
        res.append([ vertices[ vx ][ 0 ], vertices[ vx ][ 1 ], vertices[ vx ][ 2 ]])
    
    return res

mesh_vertices = quads_vertices()

console_log( mesh_vertices )

[[1.0, 0, 0], [0, 0, 0], [0, 1.0, 0], [1.0, 1.0, 0], [0, 1.0, 1.0], [1.0, 1.0, 1.0], [0, 0, 1.0], [1.0, 0, 1.0]]


In [279]:
def edges_GetTri( edge1, edge2 ):
    vxs = [ -1, -1, -1, -1 ]

    # quads keep edges on a one-based array because negative indicates
    # that we have to consider the vertices in reverse order in that edge.

    id = edge1 if edge1 > 0 else -1 * edge1
    vxs[ 0 ] = edges[ id ][ "start" ] if edge1 > 0 else edges[ id ][ "end" ]
    vxs[ 1 ] = edges[ id ][ "end" ] if edge1 > 0 else edges[id][ "start" ]

    id = edge2 if edge2 > 0 else -1 * edge2
    vxs[ 2 ] = edges[ id ][ "start" ] if edge2 > 0 else edges[ id ][ "end" ]
    vxs[ 3 ] = edges[ id ][ "end" ] if edge2 > 0 else edges[ id ][ "start" ];

    # edges must share a vertex and be informed in correct order
    if vxs[ 1 ] != vxs[ 2 ]:
        return []
    else:
        # one based to zero based
        return [ vxs[ 0 ] - 1, vxs[ 1 ] - 1, vxs[ 3 ] - 1 ]


def quads_GetTris():
    res = []
    for i in range( 0, len( quads )):
        quad = quads_rev[ i ]

        res.append( edges_GetTri( quad[ 0 ][ 0 ], quad[ 0 ][ 1 ] )) #quad.t1.e1, quad.t1.e2 ))
        res.append( edges_GetTri( quad[ 1 ][ 0 ], quad[ 1 ][ 1 ] )) #quad.t2.e3, quad.t2.e4 ))

    return sum( res, []) #.SelectMany( t => new[] { t }).ToArray().SelectMany(x => x).ToArray();

    
mesh_triangles = quads_GetTris()

console_log( mesh_triangles )

mesh_vertices = [( vx[ 0 ], vx[ 1 ], vx[ 2 ]) for vx in mesh_vertices ]
console_log( mesh_vertices )


[0, 1, 2, 2, 3, 0, 3, 2, 4, 4, 5, 3, 5, 4, 6, 6, 7, 5, 7, 6, 1, 1, 0, 7, 1, 6, 4, 4, 2, 1, 7, 0, 3, 3, 5, 7]
[(1.0, 0, 0), (0, 0, 0), (0, 1.0, 0), (1.0, 1.0, 0), (0, 1.0, 1.0), (1.0, 1.0, 1.0), (0, 0, 1.0), (1.0, 0, 1.0)]


In [280]:
def vertices_Get( parent_edge, t, pos ):
    edge = parent_edge if parent_edge > 0 else -1 * parent_edge
    
    key = { "edge": edge, "t": t, "pos": pos }

    if str( key ) in edge_to_vcs:
        return edge_to_vcs[ str( key )]
    else:
        return 0


In [281]:
def quads_GetShared( edgeid ):
    if edgeid in edges_quads:
        return edges_quads[ edgeid ]
    else:
        return ( -1, -1 )

    
def raise_error( msg ):
    print( msg )

In [282]:
def vertices_AddDemi( vertex, parent_edge, t, pos ):
    edge = parent_edge if parent_edge > 0 else -1 * parent_edge
    
    if edge == 0:
        raise_error( "Vertices must always belong to edges ( parent_edge cannot be zero )" )
        return None

    if t <= 0:
        raise_error( "t must be positive and greater than zero (num of divisions of an edge)" )
        return None

    if pos < 0 or pos > t:
        raise_error( "pos must be in [ 0, t ]" )
        return None

    # check if a vx at that position already exists, and promote it
    # to real vx if it is not one already and all edges sharing it are real edges
    vx_key = { "edge": edge, "t": t, "pos": pos }

    _exists = vertices_Get( parent_edge, t, pos )
    ikey = _exists if _exists > 0 else len( vertices ) + 1

    demi = 3 
    
    if _exists > 0:    
        # the vertice exists, chech if it can be promoted 
        # The vc can be promoted if it belongs to a demi edge and we try to create
        # the same divisions on the second quad that edge shares.
        
        print( "demi?", vertices[ ikey ][ demi ])
        
        if vertices[ ikey ][ demi ] == True:
            quads = quads_GetShared( edge )

            if quads[ 1 ] == -1:
                vx = vertices[ ikey ]
                vertices[ ikey ] = ( vx[ 0 ], vx[ 1 ], vx[ 2 ], False )

        return ikey

    # if the vx is new, add it as a demivertice (it is part of a demi edge)
    vc = vertex

    vertices[ ikey ] = ( vc.x, vc.y, vc.z, True )
    vx_to_edges[ ikey ] = [ vx_key ]

    # a new demiedge is always added when a new demivc is
    edge_to_vcs[ str( vx_key )] = ikey

    return -1 * ikey


In [283]:
def vertices_AddParent( vx, edge, t, pos ):
    vx_key = { "edge": edge, "t": t, "pos": pos }
    
    if vx_to_edges[ vx ].count( vx_key ) == 0:
        vx_to_edges[ vx ].append( vx_key )

    if str( vx_key ) not in edge_to_vcs:
        edge_to_vcs[str( vx_key )] = vx


In [284]:
def vertices_RemoveParent( next, edgeid, t, pos ):
    vx_key = { "edge": edgeid, "t": t, "pos": pos }

    while vx_to_edges[ vx ].count( vx_key ) > 0:
        vx_to_edges[ vx ].remove( vx_key )
        
    if str( vx_key ) in edge_to_vcs:
        del edge_to_vcs[ str( vx_key )]
    
    

In [285]:
def edges_Add( start, end, parent = 0, t = 1 ):
    # vcs always > 0
    if start <= 0 or end <= 0:
        raise_error( "indices must be greater than zero" )
        return

    # vertices must exist
    if start > len( vertices ) or end > len( vertices ):
        raise_error( "invalid index (inexistent vertex)" )
        return

    # a demi edge is not always a child of another, but a hard edge
    # always declare itself as a parent
    demi = 3
    _demi = False
    
    if vertices[ start ][ demi ] == True or vertices[ end ][ demi ] == True:
        _demi = True

    newid = len( edges ) + 1
    
    if _demi == False and parent == 0:
        parent = newid
        
    if parent < 0:
        parent = -1 * parent
        
    # if the edge being added is a real one, it have to inform itself as parent
    if _demi == False and parent != ( len( edges ) + 1 ):
        raise_error( "hard edges cannot have parents" )
        return

    _edge = { "start": start, "end": end, "parent": parent, "t": t }

    if str( _edge ) not in edges_rev:
        edges[ len( edges ) + 1 ] =  _edge
        edges_rev[ str( _edge )] = len( edges )
        
    if parent != newid:
        if parent not in edges_children:
            edges_children[ parent ] = []
            
        edges_children[ parent ].append({ "edge": newid, "t": t })


In [286]:
def edges_Delete( edgeid ):
    if edgeid in edges:
        _edge = edges[ edgeid ]
        del edges[ edgeid ]
        
        if str( _edge ) in edges_rev:
            del edges_rev[ str( _edge )]
            
        # if we delete an edge, all edges declaring this one as parent
        # must now be parent of themselves
        
        for edge in edges:
            if edges[ edge ][ "parent" ] == edgeid:
                print( "parenting to deleted edge " + str( edgeid ) + ": " + str( edges[ edge ]))


In [287]:
def vertice_isReal( vxid ):
    demi = 3
    return False if vertices[ vxid ][ demi ] == True else True


In [288]:
from vpython import *

def edges_Subdivide( edgeid, slices, quad ):
    
    if edges_quads[ edgeid ][ 1 ] == -1:
        # if the edge were already subdivided in a previous operation we already know the quad being 
        # divided
        if edges_quads[ edgeid ][ 0 ] != quad:
            err = "Cannot subdiv edge " + str( edgeid ) + " in quad " + str( quad )
            err = err + ". (quad already subdivided?)"

            raise_error( err )
            return

        # if the edge were already subdivided in a previous operation we already know the quad being 
        # divided
        if edges_children[ edgeid ][ 0 ][ "t" ] != slices:
            err = "Cannot subdiv edge " + str( edgeid ) + " in quad " + str( quad )
            err = err + ". (quad already subdivided?)"

            raise_error( err )
            return
    
    start = edges[ edgeid ][ "start" ] if edgeid > 0 else edges[ -1 * edgeid ][ "end" ]
    end = edges[ edgeid ][ "end" ] if edgeid > 0 else edges[ -1 * edgeid ][ "start" ]

    vx_start = vertices[ start ]
    vx_end = vertices[ end ]

    # create a list with intermediate vertices (generate ones if they don´t exist,
    # promote to real vx if they exist
    vcs = []
    for i in range( 0, slices + 1 ):
        vcs.append( 0 )
        
    vcs[ 0 ] = start
    vcs[ slices ] = end

    q = vector( vx_end[ 0 ], vx_end[ 1 ], vx_end[ 2 ]) - vector( vx_start[ 0 ], vx_start[ 1 ], vx_start[ 2 ])

    # add new vertice as demivertices 
    # (vxs deal with their own promotion to real verticies if needed)

    t = slices
    for pos in range( 0, slices + 1 ):
        if vcs[ pos ] == 0:
            # we add the vertice anyway, because this function promotes a demivx to a real vx if needed
            v3_prox = vector( vx_start[ 0 ], vx_start[ 1 ], vx_start[ 2 ]) + q / t * pos;
            vx = vertices_AddDemi( v3_prox, edgeid, t, pos )
            if vx < 0:
                vx = -1 * vx
                
            vcs[ pos ] = vx
    

    # if we are in the 2nd subdiv of this edge, subdivs have already been made in the past,
    # but we have to promote de edges to hard edges
    if edges_quads[ edgeid ][ 1 ] != -1:

        # 1st time this edge is subdivided: create child edges
        
        _edges = []
        newid = len( edges ) + 1

        for i in range( 0, slices ):
            # parent edges
            _edges.append({ "id": edgeid, "edge": ( vcs[ i ], vcs[ i + 1 ], edgeid, t, i )})

            # children
            _edges.append({ "id": newid, "edge": ( vcs[ i ], vcs[ i + 1 ], newid, 1, 0 )})
            edges_Add( vcs[ i ], vcs[ i + 1 ], edgeid, t )
            newid = newid + 1

        vx1 = 0
        vx2 = 1
        _t = 3
        pos = 4

        for _edge in _edges:
            edge = _edge[ "edge" ]

            vertices_AddParent( edge[ vx1 ], _edge[ "id" ], edge[ _t ], edge[ pos ])
            vertices_AddParent( edge[ vx2 ], _edge[ "id" ], edge[ _t ], edge[ pos ] + 1 )

        res = []
        for _edge in _edges:
            if _edge[ "id" ] != edgeid:
                res.append( _edge[ "id" ])
                
        return res
    
    else:
        
        # 2nd divisions: promote edges to real
        # remove in vx_to_edges and edge_to_vcs the divisions of the original edge

        # ( in vx_to_edges a record were added to relate the parent edge to the vxs,
        #  in the form parent_edges, divisions, position - since edges will be 
        #  promoted to hard ones and have no parent anymore, and the parent edge will
        #  be deleted, this records have to be removed)
        for vc in vcs:
            to_del = []
            rev_del = []
            
            for edge in vx_to_edges[ vc ]:
                if edge[ "edge" ] == edgeid and edge[ "t" ] == t:
                    # mark to deletion
                    to_del.append( edge )
                    rev_del.append( str( edge ))
            
            for edge in to_del:
                vx_to_edges[ vc ].remove( edge )
                
            for edge in rev_del:
                del edge_to_vcs[ edge ]

        #in edges and edges_rev the edges parented to the original edge are parented to themselves
        _edges = []
        for i in range( 0, len( vcs ) - 1 ):
            key = "{'start': " + str( vcs[ i ]) + ", 'end': " + str( vcs[ i + 1 ]) + ", 'parent': " 
            
            before = key + str( edgeid ) + ", 't': " + str( t ) + "}"
            _edge = edges_rev[ before ]
            _edges.append( _edge )
            
            after = key + str( _edge ) + ", 't': 1}" 
            del edges_rev[ before ]
            edges_rev[ after ] = _edge
            
        for _edge in _edges:
            edges[ _edge ][ "parent" ] = _edge
            edges[ _edge ][ "t" ] = 1
        
        #the original edge is removed
        _edge = edges[ edgeid ]
        del edges[ edgeid ]
        key = str( _edge )
        del edges_rev[ key ]
        
        return _edges

               

In [289]:
def vertices_GetEdges( vertex ):
    if vertex in vx_to_edges:
        return vx_to_edges[ vertex ]
    else:
        return []

In [290]:
def edges_SelectEdgesFromVcs( vcs ):
    
    _edges = []
    
    for vx in vcs:
        _edges.extend( vx_to_edges[ vx ])
        

    edict = { item[ "edge" ]: True for item in _edges }
    _edges = [ item for item in edict.keys()]
    
    return _edges
    

In [291]:
import json

# we do not iterate over all edges to find the children, the data structures allow
# something equivalent as an indexing of children in a database

def edges_SelectChildren( edgeid, divisions ):
    edge_id = edgeid if edgeid > 0 else -1 * edgeid
    t = divisions
    
    v1 = edges[ edge_id ][ "start" ]
    v2 = edges[ edge_id ][ "end" ]

    children = edges_SelectEdgesFromVcs([ v1, v2 ])
    children = { item: True for item in children if item != edgeid }
    children = { item: True for ( item, v ) in children.items() 
                    if edges[ item ][ "parent" ] == edgeid and edges[ item ][ "t" ] == t }
    
    old_vcs = { v1: True, v2: True }
                
    while len( children ) < divisions:
        vcs_d = {}
        
        for edge in children:
            vcs_d[ edges[ edge ][ "start" ]] = True
            vcs_d[ edges[ edge ][ "end" ]] = True
            
        vcs = [ item for item in vcs_d.keys() if item not in old_vcs ]
        siblings = edges_SelectEdgesFromVcs( vcs )

        siblings = { item: True for item in siblings if item != edgeid }
        siblings = { item: True for ( item, v ) in siblings.items() 
                        if edges[ item ][ "parent" ] == edgeid and edges[ item ][ "t" ] == t }

        children = {**children, **siblings }
        

    return [ item for item in children.keys()]


In [292]:
import math

def edges_FindConnected( candidates, src_edge, angle, _debug = False ):
    _edge = src_edge if src_edge > 0 else -1 * src_edge
    src = edges[ _edge ]
    
    if _debug == True:
        print( "testing edge " + str( src_edge ) + ": " + str( src ))

    start = src[ "start" ] if src_edge > 0 else src[ "end" ]
    end = src[ "end" ] if src_edge > 0 else src[ "start" ]

    if _debug == True:
        print( "candidates: " + str( candidates ))
        print( "starting vx: " + str( start ))
        print( "end vc: " + str( end ))
        
    for candidate in candidates:
        # one vx must match
        _cand = candidate if candidate > 0 else -1 * candidate
        test = edges[ _cand ]

        if _debug == True:
            print( "match against: " + str( test ))
            print( "vcs of tgt ", test[ "start" ], test[ "end" ])
        
        # connected edges must share a vertex. Here we dont enforce 
        # correct orientation
        if end == test[ "start" ] or end == test[ "end" ]:

            if _debug == True:
                print( "common vx found" )

            # calculate angle
            v1 = vertices[ start ]
            v2 = vertices[ end ]

            if _debug == True:
                print( "third vertice" )
                print( "vertices[ test[ 'start' ]] == v2? " + str( vertices[ test[ "start" ]] == v2 ))

            # three points, ignoring the point in common
            v3 = vertices[ test[ "end" ]] if vertices[ test[ "start" ]] == v2 else vertices[ test[ "start" ]]

            x, y, z = 0, 1, 2 
            
            dot = v1[ x ] * v2[ x ] * v3[ x ] + v1[ y ] * v2[ y ] * v3[ y ] + v1[ z ] * v2[ z ] * v3[ z ]
            det = v1[ x ] * (v2[ y ] * v3[ z ] - v2[ z ] * v3[ y ]) - v1[ y ] * (v2[ x ] * v3[ z ] - v2[ z ] * v3[ x ])
            det = det + v1[ z ] * (v2[ x ] * v3[ y ] - v2[ y ] * v3[ x ])

            _angle = math.atan2( det, dot )

            if _debug == True:
                print( "v1, v2, v3 ", v1, v2, v3 )
                print( "dot: " + str( dot ))
                print( "det: " + str( det ))
                print( "angle: " + str( _angle ))

            # tolerance
            if math.fabs( angle - _angle ) < 0.01: 
                return candidate

    return 0
        

In [293]:
def vertices_FromEdges( edge_list ):
    vcs = {}
    
    for edge in edge_list:
        vcs[ edges[ edge ][ "start" ]] = True
        vcs[ edges[ edge ][ "end" ]] = True
        
    return[ item for item in vcs.keys()]


In [294]:
import math

def vector_distance( v1, v2 ):
    return math.sqrt(( v1.x - v2.x )**2 + ( v1.y - v2.y )**2 + ( v1.z - v2.z )**2 )


def edge_IsEdgeVx( vx_pos, edgeid, divisions, debug = False ):
    x,y,z = 0,1,2
    
    if debug == True:
        print( "is vx over edge? (vc already exists?)" )
        print( "pos in test: " + str( vx_pos ))
        print( "edge: ", edgeid )
        
    vcidx = edges[ edgeid ][ "start" ]
    vc = vertices[ vcidx ]

    if debug == True:
        print( "distance to vx " + str( vcidx ))
        print( vector_distance( vx_pos, vector( vc[ x ], vc[ y ], vc[ z ])))
    
    if vector_distance( vx_pos, vector( vc[ x ], vc[ y ], vc[ z ])) < 0.01:
        return vcidx

    vcidx = edges[ edgeid ][ "end" ]
    vc = vertices[ vcidx ]

    if debug == True:
        print( "distance to vx " + str( vcidx ))
        print( vector_distance( vx_pos, vector( vc[ x ], vc[ y ], vc[ z ])))

    if vector_distance( vx_pos, vector( vc[ x ], vc[ y ], vc[ z ])) < 0.01:
        return vcidx

    if debug == True:
        print( "not over main vcs, testing the children" )
    
    # then demivcs of eventual children
    children = edges_SelectChildren( edgeid, divisions )
    
    vcs = vertices_FromEdges( children )
    
    for vx in vcs:
        vc = vertices[ vx ]

        if debug == True:
            print( "distance to vx " + str( vx ))
            print( vector_distance( vx_pos, vector( vc[ x ], vc[ y ], vc[ z ])))

        if vector_distance( vx_pos, vector( vc[ x ], vc[ y ], vc[ z ])) < 0.01:
            return vcidx

    return 0


In [295]:
def ToVector( edgeid ):
    id = edgeid if edgeid > 0 else -1 * edgeid

    edge = edges[ id ]
    start = vertices[ edge[ "start" ]] if edgeid > 0 else vertices[ edge[ "end" ]]
    end = vertices[ edge[ "end" ]] if edgeid > 0 else vertices[ edge[ "start" ]]

    x, y, z = 0, 1, 2 

    v3s = vector( start[ x ], start[ y ], start[ z ])
    v3e = vector( end[ x ], end[ y ], end[ z ])

    return v3e - v3s


def vertices_AddReal( vx, parent_edge, start ):
    ikey = len( vertices ) + 1 

    vertices[ ikey ] = ( vx.x, vx.y, vx.z, False )
    key = { "edge": parent_edge, "t": 1, "pos": 0 if start == True else 1 }
    vx_to_edges[ ikey ] = [ key ]

    # a new edge is always added when a new real vx is
    edge_to_vcs[ str( key )] = ikey

    return len( vertices ) 
    

def edges_AddConnected( edgeid, angle, ref_tri, divisions = 1, debug = False ):
    edge1, edge2 = 0, 1
    
    v3tri_1 = ToVector( ref_tri[ edge1 ])
    v3tri_2 = ToVector( ref_tri[ edge2 ])

    if debug == True:
        print( "ref tri vectors, edges " + str( ref_tri[ edge1 ]) + " and " + str( ref_tri[ edge2 ]))
        print( v3tri_1 )
        print( v3tri_2 )
        
    n = v3tri_1.cross( v3tri_2 )

    if debug == True:
        print( "normal:" )
        print( n )
        
    edge = ToVector( edgeid )
    nedge = edge.rotate( angle, axis=n )

    if debug == True:
        print( "tgt edge (" + str( edgeid ) + ") to vector: ", edge )
        print( "rotated edge: ", nedge )

    # with the new vector for the new edge, calculate the position 
    # of a new vx
    if edgeid > 0:
        end = edges[ edgeid ][ "end" ]
    else:
        end = edges[ -1 * edgeid ][ "start" ]
        
    v3end = ToVector( end )

    if debug == True:
        print( "ref point:", v3end )

    nvx = v3end + nedge

    if debug == True:
        print( "new vx to be added:", nvx )

    # check if the vertice already exists
    vc = edge_IsEdgeVx( nvx, ref_tri[ edge1 ], divisions, debug )
    
    if vc == 0:
        vc = edge_IsEdgeVx( nvx, ref_tri[ edge2 ], divisions, debug )
            
    if vc == 0:
        invx = vertices_AddReal( nvx, len( edges ) + 1, False )
    else:
        
        if debug == True:
            print( "found vertice that already exists" )
        invx = vc
        
    edges_Add( end, invx, len( edges ) + 1 )
    return len( edges )


In [296]:
def quad_Add( e1, e2, e3, e4 ):

    if (( e1, e2 ), ( e3, e4 )) not in quads:
        key = len( quads )

        quads[(( e1, e2 ), ( e3, e4 ))] = key
        quads_rev[ key ] = (( e1, e2 ), ( e3, e4 ))

        # since records edge->quad are created ( quad, -1 ) and
        # then the only valid operation for them in the future
        # is to add another quad shared by that edge, this 
        # part of the procedure ends up being very simple
        
        _edges = [ e1, e2, e3, e4 ]
        
        for edge in _edges:
            if edge not in edges_quads:
                edges_quads[ edge ] = ( key, -1 )
            else:
                if edges_quads[ edge ][ 0 ] != key:
                    edges_quads[ edge ] = ( edges_quads[ edge ][ 0 ], key )


In [297]:
def Select():
    # quads, unlike edges and vertices, is a zero based array
    return [ id for id in range( len( quads ))]

    
def Subdivide( selection, divisions = "2" ):
    # if selected other than quads, select all quads
    if selection[ "type" ] != "quads":
        selection = { "type": "quads", "items": Select()}

    _divisions = int( divisions )

    # subdivide selected quads
    for i in range( 0, len( selection[ "items" ])):
        quad = quads_rev[ selection[ "items" ][ i ]]

        # 1 - subdivide all edges
        quadid = selection[ "items" ][ i ]
        
        start_edges = edges_Subdivide( quad[ 0 ][ 0 ], _divisions, quadid )
        edges_2 = edges_Subdivide( quad[ 0 ][ 1 ], _divisions, quadid )
        edges_3 = edges_Subdivide( quad[ 1 ][ 0 ], _divisions, quadid )
        edges_4 = edges_Subdivide( quad[ 1 ][ 1 ], _divisions, quadid )

        new_edges = []
        
        for j in range( 0, divisions ):
            for _edge in start_edges:
                print( "start edges: " + str( start_edges ))
                
                # bottom edges
                e1 = _edge

                # we will create the two tris that compose the "quad"
                # (the tris don't have to be planar)
                # When we create a tri we normally have one edge and have to create
                # the other. However, in some cases we already have both edges
                # (if we are next to two of the original edges that compose the original quad)
                # To see if that's the case, we check if there is, in relation to the current one,
                # an existing and already connected edge 

                # 270 degrees because the angle uses the edge given as reference
                e2 = edges_FindConnected( edges_2, e1, 270.0 )
                
                print( "e2 found? " + str( e2 ))

                if e2 == 0:
                    # to add a new edge however we need to inform a reference tri so
                    # the algorithm can correctly position the edge (because the
                    # vxs or edges of the original quad being subdivided could have been moved, 
                    # and in this case the "quad" can be in reality two tris that dont have to be planar)
                    e2 = edges_AddConnected( e1, 270.0, quad[ 0 ], _divisions )
                    
                    print( "no, added: ", e2 )

                # we dont have to pass a referece tri to FindConnected because the angle between
                # two connected edges in a quad is the same regardles of its tris orientation
                e3 = edges_FindConnected( edges_3, e2, 270.0 )

                print( "e3 found? " + str( e3 ))

                if e3 == 0:
                    e3 = edges_AddConnected( e2, 270.0, quad[ 1 ], _divisions, True )

                    print( "no, added: ", e3 )

                # next row of edges for the next row of quads must be used reversed
                new_edges.append( -1 * e3 )

                e4 = edges_FindConnected( edges_4, e3, 270.0 )

                print( "e4 found? " + str( e4 ))

                if e4 == 0:
                    e4 = edges_AddConnected( e3, 270.0, quad[ 1 ], _divisions )

                    print( "no, added: ", e4 )

                quad_Add( e1, e2, e3, e4 )

            next = edges[ new_edges[ 0 ] if new_edges[ 0 ] > 0 else -1 * new_edges[ 0 ]]
            test = edges[ edges_3[ 0 ]]

            # if we reach the top of the quad (top edges of the last added row of quads == edges
            # of the original quad subdivided top edge ) the work is done
            the_same = False
            if next[ "start" ] == test[ "start" ] and next[ "end" ] == test[ "end" ]:
                the_same = True
                
            if the_same == True and next[ "parent" ] != test[ "parent" ]:
                the_same = False
            
            if the_same == False and next[ "start" ] == test[ "end" ] and next[ "end" ] == test[ "start" ]:
                the_same = True
                
            if the_same == True and next[ "parent" ] != test[ "parent" ]:
                the_same = False

            if the_same == True:
                break
                
            start_edges = new_edges.copy()
            new_edges = []

        # remove the old quad
        quad_idx = quads[ quad ]
        del quads[ quad ]
        del quads_rev[ quad_idx ]

        # TODO: remover depois dos testes
        break
        
Subdivide({ "type": None }, 3 )                     

start edges: [13, 14, 15]
e2 found? 0
no, added:  25
e3 found? 0
ref tri vectors, edges 3 and 4
<1, 0, 0>
<0, -1, 0>
normal:
<0, 0, -1>
tgt edge (25) to vector:  <0.00520602, -0.058682, 0>
rotated edge:  <0.0154554, -0.056849, 0>
ref point: <0, 0.333333, 0>
new vx to be added: <0.0154554, 0.276484, 0>
is vx over edge? (vc already exists?)
pos in test: <0.0154554, 0.276484, 0>
edge:  3
distance to vx 3
0.7236807100093212
distance to vx 4
1.221803135776245
not over main vcs, testing the children
distance to vx 3
0.7236807100093212
distance to vx 13
0.7902665744344731
distance to vx 14
0.9734222976280233
distance to vx 4
1.221803135776245
is vx over edge? (vc already exists?)
pos in test: <0.0154554, 0.276484, 0>
edge:  4
distance to vx 4
1.221803135776245
distance to vx 1
1.0226297459601095
not over main vcs, testing the children
distance to vx 4
1.221803135776245
distance to vx 15
1.059042135743601
distance to vx 16
0.9861844706698092
distance to vx 1
1.0226297459601095
no, added:  26
e

In [103]:
def plot():

    ctr = vec( 150, 50, 150 )

    # This is needed in Jupyter notebook and lab to make programs easily rerunnable
    scene = canvas( center=ctr, background = color.gray( 0.8 ))
    scene.autoscaling = False
    scene.forward = vec( 0, 2, 0 )
    scene.up = vec( 0, 0, 1 )
    scene.camera.pos = vec( 1.5, -3, 1.5 )
    scene.range = 0.6

    verts = []

    mesh_vertices = quads_vertices()
    mesh_vertices = [( vx[ 0 ], vx[ 1 ], vx[ 2 ]) for vx in mesh_vertices ]


    for vx in mesh_vertices:
        _pos = vector( vx[ 0 ], vx[ 1 ], vx[ 2 ])
        verts.append( vertex( pos = _pos, color = _pos ))

    tris = []
    mesh_triangles = quads_GetTris()

    for i in range( 0, len( mesh_triangles ), 3 ):
        _v0 = verts[ mesh_triangles[ i ]]
        _v1 = verts[ mesh_triangles[ i + 1 ]]
        _v2 = verts[ mesh_triangles[ i + 2 ]]

        tris.append( triangle( v0 = _v0, v1 = _v1, v2 = _v2 ))


Expected evolution of data:

after

edges_Subdivide( 1, 3 )

In [26]:
edges_Subdivide( 1, 3, 0 )

[13, 14, 15]

Demi-vertices 9 and 10 must be added on edge 1.<br/>
Vxs from:<br/>

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False)}

to

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False), 9: (0.6666666666666667, 0.0, 0.0, True), 10: (0.33333333333333337, 0.0, 0.0, True)}


In [87]:
print( vertices )

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False)}


In [89]:
print( vertices )

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False), 9: (0.6666666666666667, 0.0, 0.0, True), 10: (0.33333333333333337, 0.0, 0.0, True)}


in vx_to_edges 

in the vx  1 the edge {'edge':  1,: 't' 3, 'pos': 0} must be added<br/>
in the vx  1 the edge {'edge': 13,: 't' 1, 'pos': 0} must be added<br/>
in the vx  9 the edge {'edge':  1,: 't' 3, 'pos': 1} must be added<br/>
in the vx  9 the edge {'edge': 13,: 't' 1, 'pos': 1} must be added<br/>
in the vx  9 the edge {'edge': 14,: 't' 1, 'pos': 0} must be added<br/>
in the vx 10 the edge {'edge':  1,: 't' 3, 'pos': 2} must be added<br/>
in the vx 10 the edge {'edge': 14,: 't' 1, 'pos': 1} must be added<br/>
in the vx 10 the edge {'edge': 15,: 't' 1, 'pos': 0} must be added<br/>
in the vx  2 the edge {'edge':  1,: 't' 3, 'pos': 3} must be added<br/>
in the vx  2 the edge {'edge': 15,: 't' 1, 'pos': 1} must be added<br/>

from:

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12,: 't' 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}]}

to

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}, {'edge': 1, 't': 3, 'pos': 0}, {'edge': 13, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}, {'edge': 1, 't': 3, 'pos': 3}, {'edge': 15, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}], 9: [{'edge': 1, 't': 3, 'pos': 1}, {'edge': 13, 't': 1, 'pos': 1}, {'edge': 14, 't': 1, 'pos': 0}], 10: [{'edge': 1, 't': 3, 'pos': 2}, {'edge': 14, 't': 1, 'pos': 1}, {'edge': 15, 't': 1, 'pos': 0}]}


In [22]:
print( vx_to_edges )

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}]}


In [65]:
print( vx_to_edges )

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}, {'edge': 1, 't': 3, 'pos': 0}, {'edge': 13, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}, {'edge': 1, 't': 3, 'pos': 3}, {'edge': 15, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}], 9: [{'edge': 1, 't': 3, 'pos': 1}, {'edge': 13, 't': 1, 'pos': 1}, {'edge': 14, 't'

in edge_to_vcs

add:<br/>

{'edge': 1,: 't' 3, 'pos': 0} =>  vx  1<br/>
{'edge': 13,: 't' 1, 'pos': 0} => vx  1<br/>
{'edge': 1,: 't' 3, 'pos': 1} =>  vx  9<br/>
{'edge': 13,: 't' 1, 'pos': 1} => vx  9<br/>
{'edge': 14,: 't' 1, 'pos': 0} => vx  9<br/>
{'edge': 1,: 't' 3, 'pos': 2} =>  vx 10<br/>
{'edge': 14,: 't' 1, 'pos': 1} => vx 10<br/>
{'edge': 15,: 't' 1, 'pos': 0} => vx 10<br/>
{'edge': 1,: 't' 3, 'pos': 3} =>  vx  2<br/>
{'edge': 15,: 't' 1, 'pos': 1} => vx  2<br/>

from

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8}

to

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8, "{'edge': 1, 't': 3, 'pos': 1}": 9, "{'edge': 1, 't': 3, 'pos': 2}": 10, "{'edge': 1, 't': 3, 'pos': 0}": 1, "{'edge': 13, 't': 1, 'pos': 0}": 1, "{'edge': 13, 't': 1, 'pos': 1}": 9, "{'edge': 14, 't': 1, 'pos': 0}": 9, "{'edge': 14, 't': 1, 'pos': 1}": 10, "{'edge': 1, 't': 3, 'pos': 3}": 2, "{'edge': 15, 't': 1, 'pos': 0}": 10, "{'edge': 15, 't': 1, 'pos': 1}": 2}

In [23]:
print( edge_to_vcs )

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8}


In [24]:
print( edge_to_vcs )

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8, "{'edge': 1, 't': 3, 'pos': 1}": 9, "{'edge': 1, 't': 3, 'pos': 2}": 10, "{'edge': 1, 't': 3, 'pos': 0}": 1, "{'edge': 13, 't': 1

in edges<br/>
edges 13, 14 and 15 must be added, all parented to edge 1, vx 1->9, 9->10, 10->2

from

{1: {'start': 1, 'end': 2, 'parent': 1}, 2: {'start': 2, 'end': 3, 'parent': 2}, 3: {'start': 3, 'end': 4, 'parent': 3}, 4: {'start': 4, 'end': 1, 'parent': 4}, 5: {'start': 3, 'end': 5, 'parent': 5}, 6: {'start': 5, 'end': 6, 'parent': 6}, 7: {'start': 6, 'end': 4, 'parent': 7}, 8: {'start': 5, 'end': 7, 'parent': 8}, 9: {'start': 7, 'end': 8, 'parent': 9}, 10: {'start': 8, 'end': 6, 'parent': 10}, 11: {'start': 7, 'end': 2, 'parent': 11}, 12: {'start': 1, 'end': 8, 'parent': 12}}

to

{1: {'start': 1, 'end': 2, 'parent': 1, 't': 1}, 2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}, 13: {'start': 1, 'end': 9, 'parent': 1}, 14: {'start': 9, 'end': 10, 'parent': 1}, 15: {'start': 10, 'end': 2, 'parent': 1}}

In [30]:
print( edges )

{1: {'start': 1, 'end': 2, 'parent': 1, 't': 1}, 2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}}


In [156]:
print( edges )

{1: {'start': 1, 'end': 2, 'parent': 1, 't': 1}, 2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}, 13: {'start': 1, 'end': 9, 'parent': 1, 't': 3}, 14: {'start': 9, 'end': 10, 'parent': 1, 't': 3}, 15: {'start': 10, 'end': 2, 'parent': 1, 't': 3}}


in edges_rev

the record "{'start': 1, 'end': 9, 'parent': 1}": 13 must be added<br/>
"{'start': 9, 'end': 10, 'parent': 1}": 14<br/>
"{'start': 10, 'end': 2, 'parent': 1}": 15<br/>

from

{"{'start': 1, 'end': 2, 'parent': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2}": 2, "{'start': 3, 'end': 4, 'parent': 3}": 3, "{'start': 4, 'end': 1, 'parent': 4}": 4, "{'start': 3, 'end': 5, 'parent': 5}": 5, "{'start': 5, 'end': 6, 'parent': 6}": 6, "{'start': 6, 'end': 4, 'parent': 7}": 7, "{'start': 5, 'end': 7, 'parent': 8}": 8, "{'start': 7, 'end': 8, 'parent': 9}": 9, "{'start': 8, 'end': 6, 'parent': 10}": 10, "{'start': 7, 'end': 2, 'parent': 11}": 11, "{'start': 1, 'end': 8, 'parent': 12}": 12}

to


{"{'start': 1, 'end': 2, 'parent': 1, 't': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2, 't': 1}": 2, "{'start': 3, 'end': 4, 'parent': 3, 't': 1}": 3, "{'start': 4, 'end': 1, 'parent': 4, 't': 1}": 4, "{'start': 3, 'end': 5, 'parent': 5, 't': 1}": 5, "{'start': 5, 'end': 6, 'parent': 6, 't': 1}": 6, "{'start': 6, 'end': 4, 'parent': 7, 't': 1}": 7, "{'start': 5, 'end': 7, 'parent': 8, 't': 1}": 8, "{'start': 7, 'end': 8, 'parent': 9, 't': 1}": 9, "{'start': 8, 'end': 6, 'parent': 10, 't': 1}": 10, "{'start': 7, 'end': 2, 'parent': 11, 't': 1}": 11, "{'start': 1, 'end': 8, 'parent': 12, 't': 1}": 12, "{'start': 1, 'end': 9, 'parent': 1, 't': 3}": 13, "{'start': 9, 'end': 10, 'parent': 1, 't': 3}": 14, "{'start': 10, 'end': 2, 'parent': 1, 't': 3}": 15}

In [25]:
print( edges_rev )

{"{'start': 1, 'end': 2, 'parent': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2}": 2, "{'start': 3, 'end': 4, 'parent': 3}": 3, "{'start': 4, 'end': 1, 'parent': 4}": 4, "{'start': 3, 'end': 5, 'parent': 5}": 5, "{'start': 5, 'end': 6, 'parent': 6}": 6, "{'start': 6, 'end': 4, 'parent': 7}": 7, "{'start': 5, 'end': 7, 'parent': 8}": 8, "{'start': 7, 'end': 8, 'parent': 9}": 9, "{'start': 8, 'end': 6, 'parent': 10}": 10, "{'start': 7, 'end': 2, 'parent': 11}": 11, "{'start': 1, 'end': 8, 'parent': 12}": 12}


In [32]:
print( edges_rev )

{"{'start': 1, 'end': 2, 'parent': 1, 't': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2, 't': 1}": 2, "{'start': 3, 'end': 4, 'parent': 3, 't': 1}": 3, "{'start': 4, 'end': 1, 'parent': 4, 't': 1}": 4, "{'start': 3, 'end': 5, 'parent': 5, 't': 1}": 5, "{'start': 5, 'end': 6, 'parent': 6, 't': 1}": 6, "{'start': 6, 'end': 4, 'parent': 7, 't': 1}": 7, "{'start': 5, 'end': 7, 'parent': 8, 't': 1}": 8, "{'start': 7, 'end': 8, 'parent': 9, 't': 1}": 9, "{'start': 8, 'end': 6, 'parent': 10, 't': 1}": 10, "{'start': 7, 'end': 2, 'parent': 11, 't': 1}": 11, "{'start': 1, 'end': 8, 'parent': 12, 't': 1}": 12, "{'start': 1, 'end': 9, 'parent': 1, 't': 3}": 13, "{'start': 9, 'end': 10, 'parent': 1, 't': 3}": 14, "{'start': 10, 'end': 2, 'parent': 1, 't': 3}": 15}


when the 1st quad is subdiv:

- all edges are subdiv<br/>
    - vxs added dividing the parent (original) edges are demivxc declaring the new edges as parents<br/>
    - edges added dividing the parent are demiedges added declaring the original edge as parent<br/>
    - quads reference the new edges<br/>
    - original qual is removed, but its edges remains (there is one more reference to them in another quad)<br/>
    - the quad that was subdiv is removed from the 2 quads the original edge references<br/>
    
when an edge already subdiv is requested to subdiv again in the 2nd quad it shares:

- if the number of divs is different than the previous one, the operation fails<br/>
- no need to subdiv because it was already done<br/>
- demivxs declared previously are promoted to real vcs<br/>
- remove in vx_to_edges and edge_to_vcs the divisions of the original edge
- in edges and edges_rev the edges parented to the original edge are parented to themselves
- the original edge is removed


So, after the 2nd subdivision:


In [86]:
# simulating the deletion of quads
edges_quads[ 1 ] = ( 3, -1 )

edges_Subdivide( 1, 3, 3 )

add_demi <0.666667, 0, 0> 1 3 1
demi? True
add_demi <0.333333, 0, 0> 1 3 2
demi? True
[13]
[13, 14]
[13, 14, 15]


vertices 9 and 10 should be promoted to real:
    
from

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False), 9: (0.6666666666666667, 0.0, 0.0, True), 10: (0.33333333333333337, 0.0, 0.0, True)}

to

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False), 9: (0.6666666666666667, 0.0, 0.0, False), 10: (0.33333333333333337, 0.0, 0.0, False)}

In [131]:
print( vertices )


{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False), 9: (0.6666666666666667, 0.0, 0.0, False), 10: (0.33333333333333337, 0.0, 0.0, False)}


in vx_to_edges 

records must be removed:

in the vx  1 the edge {'edge':  1,: 't' 3, 'pos': 0}<br/>
in the vx  9 the edge {'edge':  1,: 't' 3, 'pos': 1}<br/>
in the vx 10 the edge {'edge':  1,: 't' 3, 'pos': 2}<br/>
in the vx  2 the edge {'edge':  1,: 't' 3, 'pos': 3}<br/>

from:

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}, {'edge': 1, 't': 3, 'pos': 0}, {'edge': 13, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}, {'edge': 1, 't': 3, 'pos': 3}, {'edge': 15, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}], 9: [{'edge': 1, 't': 3, 'pos': 1}, {'edge': 13, 't': 1, 'pos': 1}, {'edge': 14, 't': 1, 'pos': 0}], 10: [{'edge': 1, 't': 3, 'pos': 2}, {'edge': 14, 't': 1, 'pos': 1}, {'edge': 15, 't': 1, 'pos': 0}]}

to

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}, {'edge': 13, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}, {'edge': 15, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}], 9: [{'edge': 13, 't': 1, 'pos': 1}, {'edge': 14, 't': 1, 'pos': 0}], 10: [{'edge': 14, 't': 1, 'pos': 1}, {'edge': 15, 't': 1, 'pos': 0}]}

In [132]:
print( vx_to_edges )

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}, {'edge': 13, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}, {'edge': 15, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}], 9: [{'edge': 13, 't': 1, 'pos': 1}, {'edge': 14, 't': 1, 'pos': 0}], 10: [{'edge': 14, 't': 1, 'pos': 1}, {'edge': 15, 't': 1, 'pos': 0}]}


in edge_to_vcs

remove:<br/>

{'edge': 1, 't': 3, 'pos': 0} =>  vx  1<br/>
{'edge': 1, 't': 3, 'pos': 1} =>  vx  9<br/>
{'edge': 1, 't': 3, 'pos': 2} =>  vx 10<br/>
{'edge': 1, 't': 3, 'pos': 3} =>  vx  2<br/>

from

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8, "{'edge': 1, 't': 3, 'pos': 1}": 9, "{'edge': 1, 't': 3, 'pos': 2}": 10, "{'edge': 1, 't': 3, 'pos': 0}": 1, "{'edge': 13, 't': 1, 'pos': 0}": 1, "{'edge': 13, 't': 1, 'pos': 1}": 9, "{'edge': 14, 't': 1, 'pos': 0}": 9, "{'edge': 14, 't': 1, 'pos': 1}": 10, "{'edge': 1, 't': 3, 'pos': 3}": 2, "{'edge': 15, 't': 1, 'pos': 0}": 10, "{'edge': 15, 't': 1, 'pos': 1}": 2}

to

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8, "{'edge': 13, 't': 1, 'pos': 0}": 1, "{'edge': 13, 't': 1, 'pos': 1}": 9, "{'edge': 14, 't': 1, 'pos': 0}": 9, "{'edge': 14, 't': 1, 'pos': 1}": 10, "{'edge': 15, 't': 1, 'pos': 0}": 10, "{'edge': 15, 't': 1, 'pos': 1}": 2}

In [133]:
print( edge_to_vcs )

{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8, "{'edge': 13, 't': 1, 'pos': 0}": 1, "{'edge': 13, 't': 1, 'pos': 1}": 9, "{'edge': 14, 't': 1, 'pos': 0}": 9, "{'edge': 14, 't':

in edges<br/>
edges 13, 14 and 15 must be parented to themselves
edge 1 must be removed

from

{1: {'start': 1, 'end': 2, 'parent': 1, 't': 1}, 2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}, 13: {'start': 1, 'end': 9, 'parent': 1, 't': 3}, 14: {'start': 9, 'end': 10, 'parent': 1, 't': 3}, 15: {'start': 10, 'end': 2, 'parent': 1, 't': 3}}

to

{2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}, 13: {'start': 1, 'end': 9, 'parent': 13, 't': 1}, 14: {'start': 9, 'end': 10, 'parent': 14, 't': 1}, 15: {'start': 10, 'end': 2, 'parent': 15, 't': 1}}

In [158]:
print( edges )

{2: {'start': 2, 'end': 3, 'parent': 2, 't': 1}, 3: {'start': 3, 'end': 4, 'parent': 3, 't': 1}, 4: {'start': 4, 'end': 1, 'parent': 4, 't': 1}, 5: {'start': 3, 'end': 5, 'parent': 5, 't': 1}, 6: {'start': 5, 'end': 6, 'parent': 6, 't': 1}, 7: {'start': 6, 'end': 4, 'parent': 7, 't': 1}, 8: {'start': 5, 'end': 7, 'parent': 8, 't': 1}, 9: {'start': 7, 'end': 8, 'parent': 9, 't': 1}, 10: {'start': 8, 'end': 6, 'parent': 10, 't': 1}, 11: {'start': 7, 'end': 2, 'parent': 11, 't': 1}, 12: {'start': 1, 'end': 8, 'parent': 12, 't': 1}, 13: {'start': 1, 'end': 9, 'parent': 13, 't': 1}, 14: {'start': 9, 'end': 10, 'parent': 14, 't': 1}, 15: {'start': 10, 'end': 2, 'parent': 15, 't': 1}}


in edges_rev

the record "{'start': 1, 'end': 9, 'parent': 1}": 13 must be parented to itself to "{'start': 1, 'end': 9, 'parent': 13}": 13<br/>
"{'start': 9, 'end': 10, 'parent': 1}": 14 to "{'start': 9, 'end': 10, 'parent': 14}": 14<br/>
"{'start': 10, 'end': 2, 'parent': 1}": 15 to "{'start': 10, 'end': 2, 'parent': 15}": 15<br/>

from

{"{'start': 1, 'end': 2, 'parent': 1, 't': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2, 't': 1}": 2, "{'start': 3, 'end': 4, 'parent': 3, 't': 1}": 3, "{'start': 4, 'end': 1, 'parent': 4, 't': 1}": 4, "{'start': 3, 'end': 5, 'parent': 5, 't': 1}": 5, "{'start': 5, 'end': 6, 'parent': 6, 't': 1}": 6, "{'start': 6, 'end': 4, 'parent': 7, 't': 1}": 7, "{'start': 5, 'end': 7, 'parent': 8, 't': 1}": 8, "{'start': 7, 'end': 8, 'parent': 9, 't': 1}": 9, "{'start': 8, 'end': 6, 'parent': 10, 't': 1}": 10, "{'start': 7, 'end': 2, 'parent': 11, 't': 1}": 11, "{'start': 1, 'end': 8, 'parent': 12, 't': 1}": 12, "{'start': 1, 'end': 9, 'parent': 1, 't': 3}": 13, "{'start': 9, 'end': 10, 'parent': 1, 't': 3}": 14, "{'start': 10, 'end': 2, 'parent': 1, 't': 3}": 15}

to


{"{'start': 1, 'end': 2, 'parent': 1, 't': 1}": 1, "{'start': 2, 'end': 3, 'parent': 2, 't': 1}": 2, "{'start': 3, 'end': 4, 'parent': 3, 't': 1}": 3, "{'start': 4, 'end': 1, 'parent': 4, 't': 1}": 4, "{'start': 3, 'end': 5, 'parent': 5, 't': 1}": 5, "{'start': 5, 'end': 6, 'parent': 6, 't': 1}": 6, "{'start': 6, 'end': 4, 'parent': 7, 't': 1}": 7, "{'start': 5, 'end': 7, 'parent': 8, 't': 1}": 8, "{'start': 7, 'end': 8, 'parent': 9, 't': 1}": 9, "{'start': 8, 'end': 6, 'parent': 10, 't': 1}": 10, "{'start': 7, 'end': 2, 'parent': 11, 't': 1}": 11, "{'start': 1, 'end': 8, 'parent': 12, 't': 1}": 12, "{'start': 1, 'end': 9, 'parent': 13, 't': 1}": 13, "{'start': 9, 'end': 10, 'parent': 14, 't': 1}": 14, "{'start': 10, 'end': 2, 'parent': 15, 't': 1}": 15}

In [159]:
print( edges_rev )

{"{'start': 2, 'end': 3, 'parent': 2, 't': 1}": 2, "{'start': 3, 'end': 4, 'parent': 3, 't': 1}": 3, "{'start': 4, 'end': 1, 'parent': 4, 't': 1}": 4, "{'start': 3, 'end': 5, 'parent': 5, 't': 1}": 5, "{'start': 5, 'end': 6, 'parent': 6, 't': 1}": 6, "{'start': 6, 'end': 4, 'parent': 7, 't': 1}": 7, "{'start': 5, 'end': 7, 'parent': 8, 't': 1}": 8, "{'start': 7, 'end': 8, 'parent': 9, 't': 1}": 9, "{'start': 8, 'end': 6, 'parent': 10, 't': 1}": 10, "{'start': 7, 'end': 2, 'parent': 11, 't': 1}": 11, "{'start': 1, 'end': 8, 'parent': 12, 't': 1}": 12, "{'start': 1, 'end': 9, 'parent': 13, 't': 1}": 13, "{'start': 9, 'end': 10, 'parent': 14, 't': 1}": 14, "{'start': 10, 'end': 2, 'parent': 15, 't': 1}": 15}
